## Task 2

Task 2 is to develop an application to count the number of cars that go from downtown to the city centre using openCV library. 

Install the packages required. We install openCV for computer vision, numpy to handle arrays and matrices and scipy for statistics.

In [ ]:
# install openCV
!pip install opencv-python
# install numpy
!pip install numpy
# install scipy
!pip install scipy

In [2]:
 # import the libraries
import cv2
import numpy as np
import math
from scipy.stats import mode

**Note: Please place the video in the same directory of the Notebook**

In [3]:
# Capture the video
video1 = cv2.VideoCapture('Traffic_Laramie_1.mp4')
video2 = cv2.VideoCapture('Traffic_Laramie_2.mp4')

Let's create a function to calculate the centroid of a rectangle given x, y, width and height

In [4]:
# find centroids
def get_centroids(x, y, width, height):
    # find the centroids
    cent_x = (2*x + width) // 2
    cent_y = (2*y + height) // 2
    # return the centroids
    return cent_x, cent_y

We have detected the moving cars in the main street using openCV's 'createBackgroundSubtractorMOG2' algorithm in Task 1. Here, our main objective is to count the vehicles going from the city's downtown to the city centre. We use the optical flow method to find the pattern of vehicle movement and the direction between frames. Gunnar Farneback's algorithm is used to compute the dense optical flow. OpenCV's 'calcOpticalFlowFarneback' class helps us to implement this algorithm. 

Let's first write a function to help us avoid detecting and counting the same car multile times. As the car moves between frames, this is one of the major issues we have to deal with.

**The code is given below and explained in detail using line comments**

In [5]:
# The following code is inspired from the code given in analyticsvidhya.com [Reference 1].
# No code is copied directly from the reference and implemented it my own way.
# Link: https://www.analyticsvidhya.com/blog/2022/04/building-vehicle-counter-system-using-opencv/

# Declare and initialise global variables to store frame id and centroids of the cars
frame_id = 0
frame_cent = {}

# The following function helps to avoid the same car getting counted multiple times
def detect_same_obj(cars):
    """
    Find the boundary dimensions of unique cars
    Input: A list of contour area dimensions
    x, y values, width, height
    Output: Dimension and car id of unique cars in the current frame
    """
    # global variables to store frame id and centroid of the objects
    global frame_id
    global frame_cent
    
    # stores lists of bounding rectangle dimensions and frame id
    frame_dim = []
    
    # Iterate over the identified objects (cars) 
    for car in cars:
        x,y,w,h = car # store x, y values and width and height of the bounding rectangle
        cent_x, cent_y = get_centroids(x,y,w,h) # calculate the centroids of the rectangle
        # set same_car variable to false
        same_car = False 
        # iterate over the dictionary that stores frame id and centroids
        for f_id, cent in frame_cent.items():
            # find the distance between the current object's centroid and centroids in the dictionary
            euc_dist = math.hypot(cent_x - cent[0], cent_y - cent[1])
            # if the distances are closer, we can assume that the object is the same
            if euc_dist < 50:
                # update the new centroid value to frame id as the object is the same
                frame_cent[f_id] = (cent_x, cent_y)
                # store the dimensions of the bounding rectangle
                frame_dim.append([x,y,w,h,f_id])
                # set same_car to true and break out of the for loop
                same_car = True
                break
        
        # if it is a new car, add the bounding rectangle values along with frame id
        if same_car is False:
            # update the frame_cent dictionary
            frame_cent[frame_id] = (cent_x, cent_y)
            # store the frame_id and rectangle dimensions
            frame_dim.append([x,y,w,h,frame_id])
            # increase the frame id
            frame_id += 1
    # declare a new dictionary to store the objects in the current frame  
    new_frame_cent = {}
    # iterate over the objects (bounding rectangles) in the current frame
    for dim in frame_dim:
        # store the fram_id to car_id
        car_id = dim[4] # as frame_id increases by one, new car ids will be updates
        # update the new_frame_cent dictionary
        new_frame_cent[car_id] = frame_cent[car_id]
    # replace the values in frame_cent with current objects
    frame_cent = new_frame_cent.copy()
    # return the frame_dim list
    return frame_dim

Now, we use OpenCV's 'BackgroundSubtractorMOG2' algorithm to detect the cars passing through the main street. The same method we have used for Task one. The second task's objective is to count the number of cars going to the city centre from downtown. To find the direction of the moving car, we use Gunnar Farneback's algorithm (dense optical flow).

**The code is given below and explained in detail using line comments**

In [6]:
# The following code is inspired by the method discussed in medium.com and the lectures 
# No code is copied directly from from the given reference and it is written in my own way.
# [reference 1, Link: https://www.analyticsvidhya.com/blog/2022/03/vehicle-motion-detection-using-background-subtraction/]

def countCars(video):
    # background substraction - the same method used in Task 1
    bgsub_obj = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
    kernel = np.ones((3,3),np.uint8)
    # Initialise the variables to store car count
    total_cars = 0
    t_cars = 0
    cars_left = 0
    l_cars = 0
    # read and store the previous frame
    ret_prev, frame_prev = video.read()
    # resize the previous frame to half
    frame_prev_resized = cv2.resize(frame_prev, (0, 0), None, 0.5, 0.5)
    # grayscale the previous frame
    gray_prev = cv2.cvtColor(frame_prev_resized, cv2.COLOR_BGR2GRAY)

    # array of zeros as per the size of the frame
    hsv = np.zeros_like(frame_prev_resized)
    # hsv colour space - saturation value to 255
    hsv[..., 1] = 255

    # Start an infinite loop. Press button 'q' to exit
    while True:
        # Read the current video frames
        ret, frame = video.read()
        # read again if now frame is captured
        if not ret:
            continue

        # resize the current frame
        frame_resized = cv2.resize(frame, (0, 0), None, 0.5, 0.5)
        # grayscale the current frame
        gray_frame=cv2.cvtColor(frame_resized,cv2.COLOR_BGR2GRAY)

        # dense optical flow: use Gunnar Farneback's algorithm
        optical_flow = cv2.calcOpticalFlowFarneback(gray_prev, gray_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # cartToPolar metod calculates the magnitude and angle in degrees of 2D vectors.
        magnitude, angle = cv2.cartToPolar(optical_flow[:, :, 0], optical_flow[:, :, 1], angleInDegrees=True)

        # update hue - the direction
        hsv[:, :, 0] = angle/2
        # update the magnitude value - normlise to 0-255
        hsv[:, :, 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        # convert hsv colourspace to BGR
        bgr_opt = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # if the angle is greater that 10 degrees, store the angles
        move_sense = angle[magnitude > 10]
        # find the mode - most occuring angle
        move_mode = mode(move_sense)[0]

        # if move_mode is not empty, check for left movement
        if len(move_mode) > 0:
            # the following rule to identify left movement is taken from medium.com
            # reference 2, angle between 180 and 280 indicates left movement (to city center)
            if move_mode.max(0) >= 180 and move_mode.max(0) <=280:
                # update the count of cars moving to city center 
                cars_left += 1

        # Gaussian blur: Frame is colvolved with Gaussian filter
        # Removes Gaussian noise and high frequency components
        blur_frame=cv2.GaussianBlur(gray_frame,(5,5),0)

        # apply the foreground mask using createBackgroundSubtractorMOG2 object's 'apply' method
        # It will be applied on each frame
        frame_mask = bgsub_obj.apply(blur_frame)

         # Thresholding: Each pixel values is compared to the threshold - 150
        # If the pixel value is less than 150 it is set to 0 otherwise 255
        threshold_frame=cv2.threshold(frame_mask,150,255, cv2.THRESH_BINARY)[1]

        # Dilation: The kernel matrix is applied to convolve the frame
        # It increases the white region and make the object detectible
        dilated = cv2.dilate(threshold_frame, kernel, iterations = 2)

        # Contours: The findContours() method of openCV helps to find the contours in the threshold_frame
        # It uses simple aproximation method to find the endpoints of the objects
        (contours,_)=cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw a line to mark the main street
        cv2.line(frame_resized, (0, 150),(520,150),(255, 0, 0))

        detections = []
        # Iterate over the contours and mark the object
        for c in contours:
            # Draw a rectangle around the object
            (x, y, w, h)=cv2.boundingRect(c)
            # contourArea() method filters out any small contours ignore it
            # if y value is less than 150, the object is not in the main street
            if cv2.contourArea(c) < 1500 or y < 150:
                continue
            detections.append([x,y,w,h])

        # unique cars in the current frame
        boxes_ids = detect_same_obj(detections)
        # count the cars
        for box_id in boxes_ids:
            # store the bounding rectangle dimensions and car id
            x,y,w,h,id = box_id
            total_cars = id+1 # id starts with 0, so add 1
            # add the car count to frame
            cv2.putText(frame_resized, 'Total Cars: ' + str(id+1),(30,50),  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1)
            # add the count of cars moving to city center
            cv2.putText(frame_resized, 'Cars going to city center: ' + str(cars_left),(30,70),  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1)
            # add the rectangle to the resized frame
            cv2.rectangle(frame_resized, (x,y),(x+w, y+h), (0,255,0), 1)

        # show the resized frame with car counts
        cv2.imshow('frame',frame_resized)
        # hsv to bgr 
        cv2.imshow('hsv to bgr', bgr_opt)

        # update previous frame 
        gray_prev = gray_frame

        if total_cars != t_cars or cars_left != l_cars:
            print("Total cars: ", total_cars)
            print("Cars going to city center: ", cars_left)
        t_cars = total_cars
        l_cars = cars_left

        # Press 'q' to 'quit'
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    print("Total number of cars: ", total_cars)
    print("Total Cars going to city center: ", cars_left)
    # release the video capture object and close all windows        
    video.release()
    cv2.destroyAllWindows()

**Verify the app using video1**

In [ ]:
countCars(video1)

Total cars:  1
Cars going to city center:  0
Total cars:  3
Cars going to city center:  0
Total cars:  3
Cars going to city center:  1
Total cars:  3
Cars going to city center:  2
Total cars:  4
Cars going to city center:  2
Total cars:  2
Cars going to city center:  2
Total cars:  5
Cars going to city center:  2
Total cars:  5
Cars going to city center:  3
Total cars:  2
Cars going to city center:  3
Total cars:  6
Cars going to city center:  4
Total cars:  2
Cars going to city center:  4
Total cars:  2
Cars going to city center:  5
Total cars:  2
Cars going to city center:  6
Total cars:  3
Cars going to city center:  6
Total cars:  7
Cars going to city center:  6
Total cars:  8
Cars going to city center:  6
Total cars:  9
Cars going to city center:  6
Total cars:  11
Cars going to city center:  6
Total cars:  12
Cars going to city center:  6
Total cars:  13
Cars going to city center:  6
Total cars:  14
Cars going to city center:  6
Total cars:  15
Cars going to city center:  6
Total

Total number of cars: 82<br>
Total Cars going to city center: 9

**Verify the app using video2**

In [ ]:
countCars(video2)

Total cars:  1
Cars going to city center:  0
Total cars:  2
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  8
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  9
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  10
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  11
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  12
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  13
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  14
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  15
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  16
Cars going to city center:  0
Total cars:  4
Cars going to city center:  0
Total cars:  17
Cars going to city center:  0
To

Total number of cars: 73<br>
Total Cars going to city center: 5

**The the car counts shown by the application are higher than the actual number of cars as sometimes it wrongly counts the same car more than once.**

## Reference

1. Centroid Tracking<br>
https://www.analyticsvidhya.com/blog/2022/04/building-vehicle-counter-system-using-opencv/


2. Optical flow and motion detection<br>
https://medium.com/@ggaighernt/optical-flow-and-motion-detection-5154c6ba4419